In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import logging
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras import layers
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
data=pd.read_csv('Full+data.csv')
extra=pd.read_csv('extra_data.csv')
attributes=pd.read_excel('Womens+Attributes.xlsx')
USC_product_attributes=pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx')
attributes=attributes[['Style',
       '(Reference Style Lookbook, choose all that apply)']]
tags=pd.read_csv('usc_additional_tags.csv')
#Dropped all columns with ALL null values
data_dropped=data.dropna(axis=1,how='all')
#Attach additional data to full data
full_data=pd.concat([data_dropped,extra])
full_data.drop_duplicates(subset=['product_id'], keep="first",inplace = True)

## Cleaning and lemmatizing the data

In [3]:
full_data=full_data[['product_id','description','details','name','brand_category']]
full_data.fillna('UNKNOWN_TOKEN',inplace=True)
subset_for_model=full_data
subset_for_model['brand_category']=subset_for_model['brand_category'].str.replace('/',' ')
subset_for_model['brand_category']=subset_for_model['brand_category'].str.replace(',',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace('\n',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace('/',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace(',',' ')

In [4]:
subset_for_model=subset_for_model.reset_index()

In [5]:
nltk.download('stopwords')
columns=['details','description','name','brand_category']
for j in columns:
    corpus = []
    for i in range(0,len(subset_for_model)):
        new=re.sub('[^a-zA-Z]', ' ',str(subset_for_model.loc[i,j]))
        p=[]
        for k in new.split():
            separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
            new = separated.lower()
            p.append(new)
        ls=WordNetLemmatizer()
        new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
        new = ' '.join(new)
        subset_for_model.loc[i,j]=new

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shashanktiwari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
subset_for_model['details']=subset_for_model['details'].str.replace('dressesandjumpsuits','dresses and jumpsuits')
subset_for_model['brand_category']=subset_for_model['brand_category'].str.replace('dressesandjumpsuits','dresses and jumpsuits')

In [7]:
USC_product_attributes=pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx')
new_tags=pd.concat([tags,USC_product_attributes])
new_tags=new_tags[['product_id','attribute_name','attribute_value']]

# Merging the cleaned datasets with the attribute table

In [8]:
c=subset_for_model.merge(new_tags,on='product_id',how='left')

## Selecting only the attributes which are for categories.

In [9]:
c=c[c['attribute_name']=='category']

### Cleaning the column 'attribute_value'

In [10]:
c['attribute_value']=c['attribute_value'].str.replace('&','')
c['attribute_value']=c['attribute_value'].str.lower()
c['attribute_value']=c['attribute_value'].str.replace(' ','')
c['attribute_value']=c['attribute_value'].str.replace(',','')

In [11]:
c['attribute_value'].unique()

array(['top', 'onepiece', 'bottom', 'shoe', 'sweater', 'accessory',
       'blazerscoatsjackets', 'sweatshirthoodie'], dtype=object)

### Dividing the model into input (df_cat) and output (y)

In [4]:
y=c['attribute_value']

In [5]:
df_cat=c[['description','details','name','brand_category']].reset_index().drop(columns='index')

# TFIDF

In [2]:
c=pd.read_csv('c.csv')

,Unnamed: 0,index,product_id,description,details,name,brand_category,attribute_name,attribute_value
0,25,6,01DTJCERF6F4NRZ2WSJFFA1EYS,beige stretch silk slip silk spandex dry clean...,clothing top tank camis,teah stretch silk camisole,clothing top tank camis,category,top
1,45,6,01DTJCERF6F4NRZ2WSJFFA1EYS,beige stretch silk slip silk spandex dry clean...,clothing top tank camis,teah stretch silk camisole,clothing top tank camis,category,top
2,69,8,01DVPBJ6464YKYGVAE0A1HMKGN,black velvet concealed hook zip fastening back...,clothing dress mini,layered velvet mini dress,clothing dress mini,category,onepiece
3,76,8,01DVPBJ6464YKYGVAE0A1HMKGN,black velvet concealed hook zip fastening back...,clothing dress mini,layered velvet mini dress,clothing dress mini,category,onepiece
4,107,10,01DPETNS8DB4MAR29ER6ESA7XA,accordion pleat flattering midi length faux le...,skirt,faux leather pleated midi skirt,skirt,category,bottom
...,...,...,...,...,...,...,...,...,...
6775,163389,6616,01E2KZ045BCT1Q2Z2PBC0CWRGH,casual trouser cut easy drapey fit thigh taper...,unknown,micro twill pull pant,unknown,category,bottom
6776,163405,6617,01E2KXZ0WNQ96S64TDZE3WB7CX,brushed twill give cropped wide leg pant subtl...,unknown,brushed twill crop wide leg pant,unknown,category,bottom
6777,163417,6618,01E2KXWWY6JAGENED7P62090NP,tailored soft dense knit cropped cuffed hem le...,unknown,slim crop pant,unknown,category,bottom
6778,163436,6619,01E2KZTEPE7QK1V8THASF7HWNR,flatlock stitched bias seam enhance fluid drap...,unknown,camo print silk skirt,unknown,category,bottom


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer(min_df=10)
columns=['brand_category','details','description','name']
e=pd.DataFrame()
for i in columns:
    corpus = list(df_cat[i].values)
    p = vectorizer.fit_transform(corpus)
    terms = vectorizer.get_feature_names()
    print(len(terms))
    c=pd.DataFrame(p.toarray().transpose(), index=terms)
    e=pd.concat([e,c])

108
108
1622
393


### Creating the input X for models from TF-IDF

In [7]:
X=e.T.values

In [10]:
X.shape

(6780, 2231)

# Logistic Regression

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
logreg=LogisticRegression(n_jobs=1, C=0.01)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))

accuracy 0.7640117994100295


### Checking the proportion of different categories to see how well the model performs compared to random guessing

In [17]:
(1-y.value_counts()/len(y))*100

top                    77.522124
shoe                   78.466077
bottom                 79.867257
accessory              90.560472
onepiece               91.076696
sweater                91.961652
blazerscoatsjackets    93.613569
sweatshirthoodie       96.932153
Name: attribute_value, dtype: float64

# Neural Network

In [20]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
size=[(128,64),(100,50),(50,20)]
for j in size:
    nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=j).fit(X_train, y_train)
    y_pred_RF = nn.predict(X_test)
    #print("Actual % of the data for tag",p,"is",sum(subset_for_model['attribute_value'])/len(subset_for_model)*100)
    print(accuracy_score(y_test,y_pred_RF),j)
    

0.9786135693215339 (128, 64)
0.9778761061946902 (100, 50)
0.9800884955752213 (50, 20)


# Naive Bayes 

In [21]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred_NB = classifier.predict(X_test)
accuracy_score(y_test,y_pred_NB)

0.9188790560471977

# SVM

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
SVM = svm.SVC(C=0.1, kernel='linear', degree=3, gamma='auto')
classifier=SVM.fit(X_train,y_train)
y_pred_NB = classifier.predict(X_test)
accuracy_score(y_test,y_pred_NB)

0.9756637168141593

# Word2vec

In [24]:
import gensim
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
wv.init_sims(replace=True)

In [25]:
from itertools import islice
list(islice(wv.vocab, 13030, 13050))

['Memorial_Hospital',
 'Seniors',
 'memorandum',
 'elephant',
 'Trump',
 'Census',
 'pilgrims',
 'De',
 'Dogs',
 '###-####_ext',
 'chaotic',
 'forgive',
 'scholar',
 'Lottery',
 'decreasing',
 'Supervisor',
 'fundamentally',
 'Fitness',
 'abundance',
 'Hold']

In [26]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [29]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
names=['description','name','details','brand_category']
d=pd.DataFrame()
for i in names:
    X_new = df_cat.apply(lambda r: w2v_tokenize_text(r[i]),axis=1).values
    X_word_average = word_averaging_list(wv,X_new)
    X_array=pd.DataFrame(X_word_average)
    d=pd.concat([d,X_array],axis=1)

In [30]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(d.values, y, test_size = 0.2, random_state = 0)
size=[(128,64),(100,50),(50,20),(30,10)]
for j in size:
    nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=j).fit(X_train, y_train)
    y_pred_RF = nn.predict(X_test)
    #print("Actual % of the data for tag",p,"is",sum(subset_for_model['attribute_value'])/len(subset_for_model)*100)
    print(accuracy_score(y_test,y_pred_RF),j)
    #print("% for ", i, "is",max((1-(modeling[i].sum()/len(modeling))),(modeling[i].sum()/len(modeling))),'\n')

0.9778761061946902 (128, 64)
0.9771386430678466 (100, 50)
0.9749262536873157 (50, 20)
0.9771386430678466 (30, 10)


# Deep learning Keras with Word2Vec

In [26]:
import keras
# #Generate categorical values for each tag
# one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

In [37]:
y = y.astype('category').cat.codes

### We chose 'epoch'=50 based upon the testing accuracy and the 'batch size' was kept small (10) because we had a small training set and this would increase the computation speed as it would need fewer epochs to run.

In [29]:
X_train, X_test, y_train, y_test = train_test_split(d.values, y, test_size = 0.2, random_state = 0)
input_dim = X_train.shape[1]  # Number of features
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50,verbose=False,validation_data=(X_test, y_test),batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9985
Testing Accuracy:  0.9794


# Keras with our own embeddings

### We use the Keras tokenizer to vectorize our corpus into a list of integers and this encoding is kept in the form of dictionary. We added 1 to the vocab_size because 0 is the reserved index. We also used the pad_sequence to deal with the cases of different length of words in the text sequences.

In [42]:
#Keras Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
columns=['brand_category','details','description','name']
for i in ['brand_category']:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df_cat[i])
    kii= tokenizer.texts_to_sequences(df_cat[i])
    CD=tokenizer.word_index
    maxlen = 500
    kii = pad_sequences(kii, padding='post', maxlen=maxlen)
for i in ['details','description','name']:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df_cat[i])
    p = tokenizer.texts_to_sequences(df_cat[i])# Adding 1 because of reserved 0 index
    CD={**CD,**tokenizer.word_index}
    vocab_size=len(CD)+1
    p = pad_sequences(p, padding='post', maxlen=maxlen)
    kii=np.concatenate((kii,p),axis=1)   

In [43]:
len(CD)

6307

In [44]:
kii.shape

(6780, 2000)

In [45]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50
model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 50)           315400    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 315,921
Trainable params: 315,921
Non-trainable params: 0
_________________________________________________________________


In [46]:
X_train, X_test, y_train, y_test = train_test_split(kii, y, test_size = 0.2, random_state = 0)
input_dim = X_train.shape[1]  # Number of features
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10,verbose=False,validation_data=(X_test, y_test),batch_size=5)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.2260
Testing Accuracy:  0.2294


### Our own integer based embeddings did not give good results.

# Keras with Pre-trained Glove Embeddings

In [49]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [50]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('glove.6B.300d.txt',CD, embedding_dim)

In [51]:
#69% of vocabulary is covered through pre trained embeddings
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.6900760938490805

In [52]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 50)           315400    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 50)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_16 (Dense)             (None, 8)                 88        
Total params: 315,998
Trainable params: 598
Non-trainable params: 315,400
_________________________________________________________________


In [53]:
X_train, X_test, y_train, y_test = train_test_split(kii, y, test_size = 0.2, random_state = 0)
input_dim = X_train.shape[1]  # Number of features
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20,verbose=False,validation_data=(X_test, y_test),batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9987
Testing Accuracy:  0.9838


### We got the best results using Keras with a combination of Glove and TF-IDF 

# References

*  http://dsgeek.com/2018/02/19/tfidf_vectors.html
* https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568
*  https://towardsdatascience.com/build-and-compare-3-models-nlp-sentiment-prediction-67320979de61
* https://realpython.com/python-keras-text-classification/